In [26]:
import sys

lib_dir = "/home/daniele/documents/github/ftt01/phd/share/lib"
sys.path.insert( 0, lib_dir )

In [27]:
from lib import *

In [28]:
import psycopg2

In [70]:
conn = psycopg2.connect(
    host="IP.2",
    port=5432,
    database="gis",
    user="docker",
    password="docker")

cur = conn.cursor()

# cur.execute("""INSERT INTO public.eu_it_taa_era5land_grid(
# 	id, point)
# 	VALUES (?, ?);""")

# rows = cur.fetchall()

# print("Show me the databases:\n")
# for row in rows:
#     print("   ", row)

In [30]:
input_path = "/media/windows/projects/bias_correction/data/meteo/era5/kriging/precipitation/tmp/"
output_path = input_path + "biased/"
mkNestedDir( output_path )

# variables = ['2m_temperature', 'surface_net_solar_radiation',
#              'total_precipitation', 'relative_humidity']

# lat_min = 45.00
# lat_max = 47.50
# lon_min = 9.80
# lon_max = 13.00
# basin = 'trentino_alto_adige'

lat_min = 45.67
lat_max = 47.07
lon_min = 10.39
lon_max = 12.39

In [31]:
from eccodes import *
import xarray as xr
import dask as dd

In [32]:
files_to_read = glob.glob(input_path + "*.csv")

In [33]:
for f in files_to_read:

    df = pd.read_csv(f, nrows=2)
    

In [34]:
df = df.T

In [35]:
# SELECT id FROM grid_meta WHERE x=651000 AND y=5059000

In [42]:
df[0]

Unnamed: 0    Easting
V1             651000
V2             652000
V3             649000
V4             650000
               ...   
V14369         745000
V14370         746000
V14371         742000
V14372         743000
V14373         744000
Name: 0, Length: 14374, dtype: object

In [48]:
points = []

for i in range(len(df)):
    points.append( ( df.iloc[i][0],df.iloc[i][1] ) )

In [69]:
cur.close()

In [90]:
for el in points[1:]:
    # print("""INSERT INTO public.eu_it_taa_era5land(points) VALUES (ST_GeomFromText(\'POINT({x} {y})\', 32632));""".format(x=el[0], y=el[1]))
    point = "'POINT({x} {y})'".format(x=el[0], y=el[1])
    # print(point)
    cur.execute("""INSERT INTO public.eu_it_taa_era5land(points) VALUES (ST_GeomFromText({point}, 32632));""".format(point=point))
    # cur.commit()

In [88]:
curs = conn.cursor()
curs.execute("ROLLBACK")
conn.commit()